## Data loading

In [3]:
import pandas as pd

# Load the dataset
file_path = "/home/nathan/OneDrive/GitHub/Nvidia/Data/training_data.csv"
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()

,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,2,Le test de niveau en français est sur le site ...,A1
3,3,Est-ce que ton mari est aussi de Boston?,A1
4,4,"Dans les écoles de commerce, dans les couloirs...",B1


## Data Preprocessing

In [4]:
from sklearn.preprocessing import LabelEncoder

# Check for missing values
missing_values = data.isnull().sum()

# Encode the 'difficulty' column
label_encoder = LabelEncoder()
data['difficulty_encoded'] = label_encoder.fit_transform(data['difficulty'])

# Display missing values and the first few rows after encoding
missing_values, data.head()

(id            0
 sentence      0
 difficulty    0
 dtype: int64,
    id                                           sentence difficulty  \
 0   0  Les coûts kilométriques réels peuvent diverger...         C1   
 1   1  Le bleu, c'est ma couleur préférée mais je n'a...         A1   
 2   2  Le test de niveau en français est sur le site ...         A1   
 3   3           Est-ce que ton mari est aussi de Boston?         A1   
 4   4  Dans les écoles de commerce, dans les couloirs...         B1   
 
    difficulty_encoded  
 0                   4  
 1                   0  
 2                   0  
 3                   0  
 4                   2  )

## Test set loading

In [5]:
test_set = pd.read_csv("/home/nathan/OneDrive/GitHub/Nvidia/Data/unlabelled_test_data.csv")
test_set.head()

,id,sentence
0,0,Nous dûmes nous excuser des propos que nous eû...
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...
2,2,"Et, paradoxalement, boire froid n'est pas la b..."
3,3,"Ce n'est pas étonnant, car c'est une saison my..."
4,4,"Le corps de Golo lui-même, d'une essence aussi..."


### Function of feature engineering

In [6]:
import torch
import pandas as pd
from transformers import CamembertTokenizer, CamembertModel
import spacy
import pyphen
from nltk.tokenize import word_tokenize, sent_tokenize
from lexicalrichness import LexicalRichness
import numpy as np

# Load Spacy French model
nlp = spacy.load("fr_core_news_sm")

# Initialize Camembert
tokenizer = CamembertTokenizer.from_pretrained('camembert/camembert-large')
model = CamembertModel.from_pretrained('camembert/camembert-large')

# Function to calculate features for a given text
def calculate_features(text):
    # Tokenize the text into words and sentences
    words = word_tokenize(text, language='french')
    sentences = sent_tokenize(text, language='french')

    # Initialize Pyphen for syllable counting
    dic = pyphen.Pyphen(lang='fr')

    # Compute text embeddings
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

    # Lexical Diversity Measures
    lex = LexicalRichness(text)
    mtld = lex.mtld(threshold=0.72)

    # Syntactic Complexity Measures
    doc = nlp(text)
    num_subordinate_clauses = sum(1 for sent in doc.sents for token in sent if token.dep_ in ['csubj', 'csubjpass', 'advcl'])

    return {
        'total_length': len(words),
        'average_word_length': np.mean([len(word) for word in words]),
        'ttr': len(set(words)) / len(words),
        'average_sentence_length': np.mean([len(word_tokenize(sentence, language='french')) for sentence in sentences]),
        'average_syllables_per_word': np.mean([len(dic.inserted(word).split("-")) for word in words]),
        'mtld': mtld,
        'num_subordinate_clauses': num_subordinate_clauses,
        'embeddings': embeddings.tolist()  # Convert to list for easier handling
    }


2023-11-27 11:30:20.828718: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 11:30:20.828944: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 11:30:20.831742: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 11:30:21.164418: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-27 11:30:24.724392: I tensorflow/compiler/

### Applied on train dataset

In [ ]:
# Apply the feature calculation to each text
data['features'] = data['sentence'].apply(calculate_features)

# Example of accessing features of the first text
print(data['features'][0])

# Save to csv file
data.to_csv('Cleaned_Enhanced_training.csv', index=False)

### Applied on the testset

In [ ]:
# Apply the feature calculation to each text
test_set['features'] = test_set['sentence'].apply(calculate_features)

# Example of accessing features of the first text
print(test_set['features'][0])

# Save to csv file
test_set.to_csv('Cleaned_Enhanced_test.csv', index=False)